In [1]:
import numpy as np
import scipy.sparse as sps
import h5py
from termcolor import colored
import os

def decode_str(hdf5_str):
    return hdf5_str.tobytes().decode('utf-16')

# filename = 'data/Head-and-Neck_02.mat'
filename = 'data/Prostate_CK_04.mat'
f = h5py.File(filename)
name = decode_str(f['patient']['Identifier'][:])
print(colored('Patient: ' + name, 'blue'))

folder = filename.split('.')[0]
if not os.path.exists(folder):
    os.makedirs(folder)

Patient: Prostate CK 04


In [2]:
class Region:
    def __init__(self, name):
        self.name = name
        self.D = None # Dose deposition matrix (called A in TROTS)

    def compute_dose(self, x):
        self.dose = self.D.dot(x)
        self.min = self.dose.min()
        self.mean = self.dose.mean()
        self.max = self.dose.max()

def load_rois():
    print('Loading dose deposition matrices...')
    
    rois = {}
    for ref in f['patient']['StructureNames'][:]:
        name = decode_str(f[ref[0]][:])
        rois[name] = Region(name)

    # Get the dose matrix for each ROI
    n_mats = f['data']['matrix']['A'].shape[0]
    for i in range(n_mats):
        name = decode_str(f[f['data']['matrix']['Name'][i,0]][:])

        if name in rois:
            roi = rois[name]
            A = f[f['data']['matrix']['A'][i,0]]
            
            if isinstance(A, h5py.Group): # It's a sparse matrix
                data = np.array(A['data']).ravel()
                ir = np.array(A['ir']).ravel()
                jc = np.array(A['jc']).ravel()
                n_voxels = A.attrs.get('MATLAB_sparse')
                n_beamlets = jc.size - 1
                shape = (n_voxels, n_beamlets)
                
                roi.D = sps.csc_matrix((data, ir, jc), shape=shape)
                
                print(colored(f'  {name} -> {shape} (sparse)', 'green'))
            elif isinstance(A, h5py.Dataset): # It's a dense matrix
                roi.D = A[:].T
                shape = roi.D.shape
                print(colored(f'  {name} -> {shape} (dense)', 'green'))
            else:
                print(colored(f'  {name} -> Not processed!', 'magenta'))
        else:
            print(colored(f'  {name} -> Ignored', 'yellow'))

    # Check that all regions have dose matrices
    for name, roi in rois.items():
        if roi.D is None:
            print(colored(f'  {name} -> No dose matrix!', 'red'))
    
    return rois

rois = load_rois()

Loading dose deposition matrices...
  PTV 3 mm -> (4939, 2537) (dense)
  PZ -> (1928, 2537) (dense)
  Rectum -> (4968, 2537) (dense)
  Urethra -> (1652, 2537) (dense)
  Bladder -> (4990, 2537) (sparse)
  Rectum (mean) -> Ignored
  Urethra (mean) -> Ignored
  Bladder (mean) -> Ignored
  From 30 mm to External -20 mm -> (26645, 2537) (sparse)
  Penis/Scrotum -> (4960, 2537) (sparse)
  Femoral Head (R) -> (4867, 2537) (sparse)
  Femoral Head (L) -> (4938, 2537) (sparse)
  PTV Ring 20 mm - 30 mm -> (4980, 2537) (dense)
  Smoothing Linear -> Ignored
  Smoothing Quadratic -> Ignored
  External Ring 20 mm -> (11742, 2537) (sparse)
  Penis/Scrotum (mean) -> Ignored
  PTV 7 mm -> (4538, 2537) (dense)
  Sm2 beam 1 -> Ignored
  Sm2 beam 2 -> Ignored
  Sm2 beam 3 -> Ignored
  Sm2 beam 4 -> Ignored
  Sm2 beam 5 -> Ignored
  Sm2 beam 6 -> Ignored
  Sm2 beam 7 -> Ignored
  Sm2 beam 8 -> Ignored
  Sm2 beam 9 -> Ignored
  Sm2 beam 10 -> Ignored
  Sm2 beam 11 -> Ignored
  Sm2 beam 12 -> Ignored
  Sm2 be

In [3]:
import os

def n_voxels(rois):
    n_voxels = 0
    for name, roi in rois.items():
        n_voxels += roi.D.shape[0]
    return n_voxels

def gen_m(folder):
    name = decode_str(f['patient']['Identifier'][:]).replace(' ', '_')
    with open(os.path.join(folder, 'm_' + name + '.txt'), 'w') as file:
        file.write(name + '\n')
        # We don't care about beam geometry for now.
        file.write('1 // Number of beams \n')
        file.write('1 {} \n'.format(int(f['data']['misc']['size'][0][0])))
        file.write('{} // Number of voxels \n'.format(n_voxels(rois)))
        # d_ files will hold float values without need of scaling.
        file.write('1.0 // DoseGridScaling \n'.format(n_voxels(rois)))
        file.write('{} // Number of ROIs \n'.format(len(rois)))
        for i, (name, roi) in enumerate(rois.items()):
            file.write('{} {}\n'.format(2**i, name))
            

def gen_v(folder):
    name = decode_str(f['patient']['Identifier'][:]).replace(' ', '_')
    with open(os.path.join(folder, 'v_' + name + '.txt'), 'w') as file:
        for i, (name, roi) in enumerate(rois.items()):
            print(name)
            voxel_region = 2**i
            for j in range(roi.D.shape[0]):
                file.write('{}\n'.format(voxel_region))
                
gen_m(folder)
gen_v(folder)

PTV 3 mm
Bladder
Femoral Head (R)
Femoral Head (L)
Urethra
Penis/Scrotum
Rectum
PZ
From 30 mm to External -20 mm
PTV Ring 20 mm - 30 mm
External Ring 20 mm
PTV 7 mm


In [4]:
def gen_d(folder):
    name = decode_str(f['patient']['Identifier'][:]).replace(' ', '_')
    
    total_nnz = 0
    for n, roi in rois.items():
        coo = sps.coo_matrix(roi.D)
        total_nnz += coo.nnz
    
    with open(os.path.join(folder, 'd_' + name + '.txt'), 'w') as file:
        file.write('{} \n'.format(total_nnz))
        row_offset = 0
        for i, (n, roi) in enumerate(rois.items()):
            coo = sps.coo_matrix(roi.D)
            print(f'[{i+1:02d}/{len(rois):02d}] {n}: {coo.nnz} nonzeros...')
            for j in range(coo.nnz):
                file.write('{} {} {:.8f}\n'.format(coo.row[j] + row_offset, coo.col[j], coo.data[j]))
            row_offset += coo.shape[0]

gen_d(folder)

[01/12] PTV 3 mm: 10396619 nonzeros...
[02/12] Bladder: 3008202 nonzeros...
[03/12] Femoral Head (R): 2820721 nonzeros...
[04/12] Femoral Head (L): 2332479 nonzeros...
[05/12] Urethra: 3183698 nonzeros...
[06/12] Penis/Scrotum: 82821 nonzeros...
[07/12] Rectum: 5101839 nonzeros...
[08/12] PZ: 4374377 nonzeros...
[09/12] From 30 mm to External -20 mm: 3100106 nonzeros...
[10/12] PTV Ring 20 mm - 30 mm: 5209712 nonzeros...
[11/12] External Ring 20 mm: 703396 nonzeros...
[12/12] PTV 7 mm: 8316443 nonzeros...


In [5]:
def gen_x():
    name = decode_str(f['patient']['Identifier'][:]).replace(' ', '_')
    fluence = f['solutionX'][:].ravel()
    
    with open(os.path.join(folder, 'x_' + name + '.txt'), 'w') as file:
        for x in fluence:
            file.write('{}\n'.format(x))

gen_x()